## Notes

This notebook is designed to be run in **Colab (Google Colaboratory)**, not in your local Jupyter Notebook environment.

So, if you want to run it locally, please rewrite the PATH settings of each cell and the file input/output code appropriately.

In [ ]:
#@title Install { display-mode: "form" }
%%bash
if [ ! -d af_backprop ]; then
  git clone https://github.com/sokrypton/af_backprop.git
#   cd af_backprop
#   # 7246fe5 on 25 Feb
#   git reset --hard 7246fe544e
#   cd /content
  pip -q install biopython dm-haiku==0.0.5 ml-collections py3Dmol
fi
if [ ! -d params ]; then
  mkdir params
  curl -fsSL https://storage.googleapis.com/alphafold/alphafold_params_2021-07-14.tar | tar x -C params
fi
# Latest commit fc00f95 on 21 Dec 2021
wget -qnc https://raw.githubusercontent.com/sokrypton/ColabFold/fc00f951cef535d3a643bf432f965b4eeba71b0e/colabfold/colabfold.py
# Download solubility-aware design.py
wget https://raw.githubusercontent.com/ohuelab/Solubility_AfDesign/solubility/design.py
wget https://raw.githubusercontent.com/ohuelab/Solubility_AfDesign/solubility/design_util.py
wget https://raw.githubusercontent.com/ohuelab/Solubility_AfDesign/solubility/constant.py

def color_seq_solubility(input):
    hydrophilic = ["R", "K", "D", "E", "N", "Q"]
    neutral = ["S", "G", "H", "T", "A", "P"]
    hydrophobic = ["Y", "V", "M", "C", "L", "F", "I", "W"]
    HPH='\033[094m'
    NEU='\033[092m'
    HPO='\033[090m'
    END='\033[0m'
    result = []
    for i in input:
        if i in hydrophilic:
            result.append(HPH + i + END)
        elif i in neutral:
            result.append(NEU + i + END)
        elif i in hydrophobic:
            result.append(HPO + i + END)
        else:
            result.append(i)
    print("".join(result))

In [ ]:
#@title import libraries
import sys
sys.path.append('/content/af_backprop')

import os
from google.colab import files
import numpy as np
from IPython.display import HTML
from design import mk_design_model, clear_mem
from design_util import *

%env TF_CUDNN_DETERMINISTIC=1

In [ ]:
#@title Target protein, binder settings
protocol = "binder"
solubility_index = "Solubility-Weighted Index" #@param ["Solubility-Weighted Index", "Hydropaty index", "Hydrophobicity index"]
if solubility_index == "Solubility-Weighted Index":
    _solubility_index = "swi"
elif solubility_index == "Hydropaty index":
    _solubility_index = "hyp"
elif solubility_index == "Hydrophobicity index":
    _solubility_index = "hyd"

pdb_id = "1YCR" #@param {type:"string"}
chain = "A" #@param {type:"string"}
solubility_weight = 0.5 #@param {type:"number"}
seed = 1 #@param {type:"integer"}
binder_length = 13 #@param {type:"integer"}

clear_mem()
model = mk_design_model(protocol=protocol, solubility_index=_solubility_index) 
model.prep_inputs(pdb_filename=get_pdb(pdb_id), chain=chain,
                        binder_len=binder_length)

In [ ]:
#@title Weight settings
#@markdown When the solubility weight is 0, it is the same as the default AfDesign's binder hallucination protocol.
solubility_weight = 0.5 #@param {type:"number"}
#@markdown Other weight settings by default
msa_ent = 0.01 #@param {type:"number"}
plddt = 0.1 #@param {type:"number"}
pae_intra = 0.1 #@param {type:"number"}
pae_inter = 1.0 #@param {type:"number"}
con_intra = 0.1 #@param {type:"number"}
con_inter = 0.5 #@param {type:"number"}

## Solubility-aware binder hallucination

In [ ]:
# Solubility-aware binder design
# If you want to try other settings, please refer to the README (https://github.com/blacktanktop/Solubility_AfDesign)
model.restart(seed=seed)

model.opt["weights"].update({"solubility": solubility_weight, "msa_ent": msa_ent, 
                             "plddt": plddt, "pae_intra": pae_intra, "pae_inter": pae_inter,
                             "con_intra": con_intra, "con_inter": con_inter})
print("target_length",model._target_len)
print("binder_length",model._binder_len)
print("weights",model.opt["weights"])

model.design_3stage(soft_iters=100, temp_iters=100, hard_iters=10)

## Animation

In [ ]:
ani = model.animate()
HTML(ani.to_html5_video())

In [ ]:
# download mp4 file
ani.save(f"./Solubility_AfDesign_{solubility_index}_{pdb_id}_{chain}.mp4", writer="ffmpeg",dpi=100)

## Get best sequence

In [ ]:
seq = model.get_seqs()[0]
color_seq_solubility(seq)

## Save in PDB format (target as chain A binder as chain B)

In [ ]:
save_binder(model, f"Solubility_AfDesign_{solubility_index}_{pdb_id}_{chain}.pdb")